In [1]:
nce = 27
instrument = 'QE'

var_mods = []
fix_mods = []

labeling_channels = {}

mod_map = {
    # "Cysteinyl@C": ["C[Cysteinyl]"],
    # 'Dimethyl@K':['K[Dimethyl (K)]',"K(UniMod:36)"],
    # 'Dimethyl@Any N-term':['_[Dimethyl (K)]',"_(UniMod:36)"],
}

max_var_mods = 0
min_pep_mz = 300
max_pep_mz = 1650
precursor_charge_min = 1
precursor_charge_max = 3


In [2]:
rt_ccs_refining_tsv_list = [
    "data/PXD034772/C3N-03023-DIA/C3N-03023-01/report.tsv"
]
rt_ccs_refining_tsv_type = "diann" # diann or swath
fine_tune = True
transfer_ms2 = False # always False for diann

In [3]:
from peptdeep.pretrained_models import ModelManager
from alphabase.peptide.fragment import get_charged_frag_types

frag_types = get_charged_frag_types(['b','y'], 2)


In [4]:
model_mgr = ModelManager()
model_mgr.load_installed_models()
model_mgr.nce = nce
model_mgr.instrument = instrument

In [5]:
def train():
    if transfer_ms2:
        from alphabase.spectral_library.reader import LibraryReaderBase as Reader
        lib = Reader()
    else:
        from alphabase.psm_reader.psm_reader import psm_reader_provider
        lib = psm_reader_provider.get_reader(rt_ccs_refining_tsv_type)

    lib.add_modification_mapping(mod_map)
    lib.import_files(set(rt_ccs_refining_tsv_list))
    model_mgr.train_verbose = True
    if transfer_ms2:
        model_mgr.epoch_to_train_ms2 = 50
        model_mgr.warmup_epoch_to_train_ms2 = 10
        model_mgr.psm_num_to_train_ms2 = 20000
        model_mgr.psm_num_to_test_rt_ccs = 20000
        model_mgr.train_ms2_model(lib.psm_df)
    model_mgr.epoch_to_train_rt_ccs = 100
    model_mgr.warmup_epoch_to_train_rt_ccs = 20
    model_mgr.psm_num_to_train_rt_ccs = 15000
    model_mgr.psm_num_to_test_rt_ccs = 15000
    
    model_mgr.train_rt_model(lib.psm_df)
    # if "ccs" in lib.psm_df.columns:
    #     model_mgr.train_ccs_model(lib.psm_df)
    return lib.psm_df
if fine_tune:
    psm_df = train()

2024-08-13 16:04:37> 15000 PSMs for RT model training/transfer learning
2024-08-13 16:04:37> Testing pretrained RT model:
   R_square         R     slope  intercept  test_num
0  0.978783  0.989335  0.814465   0.262977      1498
2024-08-13 16:04:37> Training with fixed sequence length: 0
[Training] Epoch=1, lr=5e-06, loss=0.18458417430520058
[Training] Epoch=2, lr=1e-05, loss=0.18344332799315452
[Training] Epoch=3, lr=1.5e-05, loss=0.17880204543471337
[Training] Epoch=4, lr=2e-05, loss=0.17138897702097894
[Training] Epoch=5, lr=2.5e-05, loss=0.15987767577171325
[Training] Epoch=6, lr=3e-05, loss=0.1450254376977682
[Training] Epoch=7, lr=3.5e-05, loss=0.12637543454766273
[Training] Epoch=8, lr=4e-05, loss=0.1011561531573534
[Training] Epoch=9, lr=4.5e-05, loss=0.07388038765639067
[Training] Epoch=10, lr=5e-05, loss=0.05448633227497339
[Training] Epoch=11, lr=5.500000000000001e-05, loss=0.04625553283840418
[Training] Epoch=12, lr=6e-05, loss=0.03895826507359743
[Training] Epoch=13, lr=6.5

In [6]:
model_mgr.save_models("model/transfer_peptdeep_model_0813")